In [1]:
import pandas as pd
import numpy as np

In [44]:
df = pd.read_csv('data/data.csv')

In [45]:
df.info(memory_usage=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11914 entries, 0 to 11913
Data columns (total 16 columns):
Make                 11914 non-null object
Model                11914 non-null object
Year                 11914 non-null int64
Engine Fuel Type     11911 non-null object
Engine HP            11845 non-null float64
Engine Cylinders     11884 non-null float64
Transmission Type    11914 non-null object
Driven_Wheels        11914 non-null object
Number of Doors      11908 non-null float64
Market Category      8172 non-null object
Vehicle Size         11914 non-null object
Vehicle Style        11914 non-null object
highway MPG          11914 non-null int64
city mpg             11914 non-null int64
Popularity           11914 non-null int64
MSRP                 11914 non-null int64
dtypes: float64(3), int64(5), object(8)
memory usage: 1.5+ MB


In [239]:
df.head()

,Make,Model,Year,Engine Fuel Type,Engine HP,Engine Cylinders,Transmission Type,Driven_Wheels,Number of Doors,Market Category,Vehicle Size,Vehicle Style,highway MPG,city mpg,Popularity,MSRP
0,BMW,1 Series M,2011,premium unleaded (required),335.0,6.0,MANUAL,rear wheel drive,2.0,"Factory Tuner,Luxury,High-Performance",Compact,Coupe,26,19,3916,46135
1,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Convertible,28,19,3916,40650
2,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,High-Performance",Compact,Coupe,28,20,3916,36350
3,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Coupe,28,18,3916,29450
4,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,Luxury,Compact,Convertible,28,18,3916,34500


- clean data

In [133]:
df.Make = df.Make.str.replace("-", " ").replace("/", "")
df.Model = df.Model.str.replace("-", " ").replace("/", "")

In [134]:
pd.DataFrame(df.Make.unique().tolist()).to_csv('data/make_list.txt', index=False, header=None)

In [135]:
pd.DataFrame(df.Model.unique().tolist()).to_csv('data/model_list.txt', index=False, header=None)

In [7]:
!ls -l

total 2000
-rw-r--r--  1 txuantu  staff   14517 Nov  8 11:26 Gen_Data.ipynb
-rw-r--r--  1 txuantu  staff   11357 Nov  8 10:18 LICENSE
-rw-r--r--  1 txuantu  staff    1053 Nov  8 10:18 Makefile
-rw-r--r--  1 txuantu  staff    4609 Nov  8 10:18 README.md
-rw-r--r--  1 txuantu  staff     808 Nov  8 10:18 actions.py
drwxr-xr-x  5 txuantu  staff     170 Nov  8 10:38 data
-rw-r--r--  1 txuantu  staff     429 Nov  8 10:18 domain.yml
-rw-r--r--  1 txuantu  staff      56 Nov  8 10:18 endpoints.yml
-rw-r--r--  1 txuantu  staff     225 Nov  8 11:47 nlu_config.yml
-rw-r--r--  1 txuantu  staff  961884 Nov  8 10:18 rasa-stack-mockup.gif
-rw-r--r--  1 txuantu  staff     604 Nov  8 10:18 requirements.txt


In [9]:
%%bash

# export LC_ALL=en_US.UTF-8
# export LANG=en_US.UTF-8
python -m rasa_nlu.train -c nlu_config.yml --data data/nlu_data.md -o models --fixed_model_name nlu --project current --verbose

/Volumes/BOOTCAMP/Tools/anaconda2.4/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
2018-11-11 14:33:45 INFO     rasa_nlu.utils.spacy_utils  - Trying to load spacy model with name 'en'
2018-11-11 14:33:46 INFO     rasa_nlu.components  - Added 'nlp_spacy' to component cache. Key 'nlp_spacy-en'.
2018-11-11 14:34:01 INFO     rasa_nlu.training_data.loading  - Training data format of data/nlu_data.md is md
2018-11-11 14:34:01 INFO     rasa_nlu.training_data.training_data  - Training data stats: 
	- intent examples: 56 (10 distinct intents)
	- Found intents: 'name', 'user_dob', 'car_model', 'affirm', 'thank', 'greet', 'pricing', 'goodbye', 'expire_date', 'year_model'
	- entity examples: 18 (3 distinct entities)
	- found entities: 'make', 'model', 'nam

- Add fallback

In [12]:
from rasa_core.policies.fallback import FallbackPolicy
from rasa_core.policies.keras_policy import KerasPolicy
from rasa_core.agent import Agent

fallback = FallbackPolicy(fallback_action_name="action_default_fallback",
                          core_threshold=0.3,
                          nlu_threshold=0.3)

agent = Agent("domain.yml", policies=[KerasPolicy(), fallback])

:0: UserWarning: You do not have a working installation of the service_identity module: 'No module named service_identity'.  Please install it from <https://pypi.python.org/pypi/service_identity> and make sure all of its dependencies are satisfied.  Without the service_identity module, Twisted can perform only rudimentary TLS client hostname verification.  Many valid certificate/hostname mappings may be rejected.
/Volumes/BOOTCAMP/Tools/anaconda2.4/lib/python2.7/site-packages/pykwalify/core.py:99: UnsafeLoaderWarning: 
The default 'Loader' for 'load(stream)' without further arguments can be unsafe.
Use 'load(stream, Loader=ruamel.yaml.Loader)' explicitly if that is OK.
Alternatively include the following in your code:

  import warnings
  warnings.simplefilter('ignore', ruamel.yaml.error.UnsafeLoaderWarning)

In most other cases you should consider using 'safe_load(stream)'
  data = yaml.load(stream)


In [14]:
training_data = agent.load_data('data/stories.md')

    
agent.train(training_data,
                   batch_size=20,
                   epochs=200,
                   validation_split=0.2,
                   max_number_of_trackers=100)

Processed trackers: 100%|██████████| 24/24 [00:01<00:00, 13.18it/s, # actions=84]


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (None, 5, 42)             0         
_________________________________________________________________
lstm (LSTM)                  (None, 32)                9600      
_________________________________________________________________
dense (Dense)                (None, 18)                594       
_________________________________________________________________
activation (Activation)      (None, 18)                0         
Total params: 10,194
Trainable params: 10,194
Non-trainable params: 0
_________________________________________________________________
Train on 67 samples, validate on 17 samples
Epoch 1/200
67/67 [==============================] - 1s 15ms/step - loss: 2.8735 - acc: 0.0597 - val_loss: 2.7838 - val_acc: 0.4118
Epoch 2/200
67/67 [==============================] 

In [ ]:
fallback = FallbackPolicy(fallback_action_name="utter_fallback",
                          core_threshold=0.2,
                          nlu_threshold=0.2)
    agent = Agent(domain_file, policies=[
        AugmentedMemoizationPolicy(max_history=max_history),
        KerasPolicy(),fallback])
    training_data = agent.load_data(training_data_file)

    
    agent.train(training_data,
                       batch_size=20,
                       epochs=200,
                       validation_split=0.2,
                       max_number_of_trackers=100)

In [3]:
from duckling import DucklingWrapper

In [4]:
d = DucklingWrapper()


In [5]:
dt = d.parse_time(u'birth date is 20Oct 2018')

In [6]:
[d['text'] for d in dt if d['value']['grain'] == u'year']

[]

In [272]:
dt

[{u'dim': u'time',
  u'end': 24,
  u'start': 14,
  u'text': u'20Oct 2018',
  u'value': {u'grain': u'day',
   u'others': [{u'grain': u'day', u'value': u'2018-10-20T00:00:00.000+08:00'}],
   u'value': u'2018-10-20T00:00:00.000+08:00'}}]

In [ ]:
import datetime as DT


In [238]:
pd.to_datetime(dt[0]['value']['value'][:10])

Timestamp('2020-10-10 00:00:00')

In [235]:
(pd.Timestamp(DT.datetime.now()) -  pd.to_datetime(dt[0]['value']['value'][:10])).days

-701

In [218]:
(pd.datetime.now() - pd.to_datetime(dt[0]['value']['value'][:10]))

Timedelta('1500 days 16:53:03.614855')

In [206]:
dt[0]['value']['value']

u'2014-10-01T00:00:00.000+08:00'

In [10]:
from rasa_nlu.model import Interpreter
import json
interpreter = Interpreter.load("./models/current/nlu")


INFO:tensorflow:Restoring parameters from ./models/current/nlu/intent_classifier_tensorflow_embedding.ckpt


In [11]:
message = u"expiration date is 20Oct 2020"
result = interpreter.parse(message)
print(json.dumps(result, indent=2))

{
  "entities": [
    {
      "extractor": "ner_duckling", 
      "confidence": 1.0, 
      "end": 29, 
      "text": "20Oct 2020", 
      "value": "2020-10-20T00:00:00.000Z", 
      "entity": "time", 
      "start": 19, 
      "additional_info": {
        "grain": "day", 
        "value": "2020-10-20T00:00:00.000Z", 
        "others": [
          {
            "grain": "day", 
            "value": "2020-10-20T00:00:00.000Z"
          }
        ]
      }
    }
  ], 
  "intent": {
    "confidence": 0.7302823066711426, 
    "name": "expire_date"
  }, 
  "text": "expiration date is 20Oct 2020", 
  "intent_ranking": [
    {
      "confidence": 0.7302823066711426, 
      "name": "expire_date"
    }, 
    {
      "confidence": 0.43963658809661865, 
      "name": "name"
    }, 
    {
      "confidence": 0.12008696794509888, 
      "name": "thank"
    }, 
    {
      "confidence": 0.03651262819766998, 
      "name": "pricing"
    }, 
    {
      "confidence": 0.0, 
      "name": "year_model"
 

In [16]:
import re

In [25]:
x = r'[0-9]{4}[-/][0-9]{2}[-/][0-9]{2}'

In [28]:
re.match(string='1000/10/100', pattern=x).group()

'1000/10/10'